# Notebook #5: Federated Evaluations

### Install the Rhino Health Python SDK, Load All Necessary Libraries and Login to the Rhino FCP

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.axes
import matplotlib.figure
import matplotlib.pyplot as plt
from PIL import Image
import os
import sys
import getpass
import json
import io
import base64

import rhino_health as rh
from rhino_health.lib.metrics import RocAuc, RocAucWithCI

In [ ]:
my_username = "FCP_LOGIN_EMAIL" # Replace this with the email you use to log into Rhino Health
session = rh.login(username=my_username, password=getpass.getpass())

### Load the Results Cohorts from the Pneumonia Training & Validation Process

In [ ]:
project = session.project.get_project_by_name("YOUR_PROJECT_NAME")  # Replace with your project name
results_cohorts = session.cohort.search_for_cohorts_by_name('COHORT_SUFFIX') # Change it with your suffix
[cohort.name for cohort in results_cohorts]

### Calculate ROC (Underlying Results Data Stays On-prem)

In [ ]:
# function to plot ROC

def plot_roc(results,cohorts):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    linestyle_cycle = ['-', '--']
    fig, ax = plt.subplots(figsize=[6, 4], dpi=200)
    linestyle = linestyle_cycle[0]
    
    for i,result in enumerate (results):
        roc_metrics = result.output
        color = colors[0]
        ax.plot(roc_metrics['fpr'], roc_metrics['tpr'], color=colors[i], 
                linestyle=linestyle, label=cohorts[i])
        ax.legend(loc='lower right')

    ax.title.set_text('ROC per Site')
    ax.set_xlabel('1 - Specificity')
    ax.set_ylabel('Sensitivity')
    ax.grid(True)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    fig.canvas.draw()
    return fig

In [ ]:
results = []
cohorts = []
report_data = []
report_data.append({"type": "Title", "data": "ROC Analysis"})

for result in results_cohorts:
    cohort = session.cohort.get_cohort(result.uid)
    cohorts.append(cohort.name.split('-')[0])
    metric_configuration = RocAuc(y_true_variable="Pneumonia",
                                  y_pred_variable="Model_Score")
    results.append(cohort.get_metric(metric_configuration))
fig = plot_roc(results, cohorts)
image_to_store = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
image_to_store.save("ROC_per_site.png", format='png', optimize=True, quality=100)

with open("ROC_per_site.png", "rb") as temp_image:
    base_64_image = base64.b64encode(temp_image.read()).decode("utf-8")
    report_data.append(
              {
                 "type": "Image",
                 "data": {
                     "image_filename": "ROC per site",
                     "image_base64": base_64_image,
                 },
                 "width": 100
              }
    )

### Upload the visualizations to the Rhino Health Platform

In [ ]:
model_result_uid = "XXXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX" # Paste the UID of the model results object for your NVF

In [ ]:
print("Sending visualizations to the Cloud")

result = session.post(f"federatedmodelactions/{model_result_uid}/set_report/", 
                      data={"report_data": json.dumps(report_data)})
print('Done')